In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import shfl
import torch
import copy
import cv2
import numpy as np

from sklearn.preprocessing import LabelBinarizer

from shfl.private import UnprotectedAccess
from CIT.model import CITModel
from utils import get_federated_data_csv, get_data_csv
from ClassifierModel import ClassifierModel

/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/frame

In [2]:
args = {"data_path":"../data/COVIDGR1.0-Segmentadas", 
        "csv_path": "../partitions/partition_iid_3nodes_1.csv",
        "output_path": "../weights",
        "input_path": "",
        "model_name":"transferlearning.model", 
        "label_bin": "lb.pickle", 
        "batch_size": 8,
        "federated_rounds": 3,
        "epochs_per_FL_round": 20,
        "num_nodes": 3,
        "size_averaging": 1,
        "random_rotation": 0,
        "random_shift": 0, 
        "random_zoom": 0,
        "horizontal_flip": False,        
        "finetune": True,
        "train_network": True}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
a = ['N', 'P']
b = ['NTN', 'NTP', 'PTP', 'PTN']
lb1 = LabelBinarizer()
lb2 = LabelBinarizer()
lb1.fit(a)
lb2.fit(b)

print("[INFO] Fetching federated data...")
federated_data, train_data, train_label, test_data, test_label, train_files, test_files, args["num_nodes"] = get_federated_data_csv(args["data_path"], args["csv_path"], lb1)
federated_data.configure_data_access(UnprotectedAccess())
print(len(train_data))
print(len(test_data))
print(args["num_nodes"])

for i in range(federated_data.num_nodes()):
    data = federated_data[i].query()._data
    print(len(data))

print("[INFO] done")

[INFO] Fetching federated data...
681
171
3
225
222
234
[INFO] done


In [4]:
def cit_builder():    
    return CITModel(['N', 'P'], classifier_name = "resnet18", folds=1, lambda_values = [0.05], batch_size=args["batch_size"], epochs=args["epochs_per_FL_round"], device=device)


In [ ]:
aggregator = shfl.federated_aggregator.FedAvgAggregator()
cit_federated_government = shfl.federated_government.FederatedGovernment(cit_builder, federated_data, aggregator)
cit_federated_government.run_rounds(args["federated_rounds"], test_data, test_label)


Accuracy round 0


  0%|          | 0/23 [00:00<?, ?it/s]

Training node 0
[INFO] weights = [1.         0.93965517]


[Validating]: Acc_D: 0.5217: 100%|██████████| 23/23 [00:00<00:00, 26.71it/s]


[INFO] Initial Valid Scores: 
Valid Acc = 0.5217391304347826
Valid Loss = 0.8046197476594344
[INFO] LAMBDA: 0.05


...

  0%|          | 0/24 [00:00<?, ?it/s]

[INFO] Summary of training for LAMBDA = 0.05 (best model values)
Valid Acc = 0.6521739130434783
Valid Loss = 0.9070882100368971
Training node 2
[INFO] weights = [1.       0.828125]


[Validating]: Acc_D: 0.4583: 100%|██████████| 24/24 [00:00<00:00, 35.53it/s]


[INFO] Initial Valid Scores: 
Valid Acc = 0.4583333333333333
Valid Loss = 0.8612320696314176
[INFO] LAMBDA: 0.05


[Validating]: Acc_D: 0.4583: 100%|██████████| 24/24 [00:00<00:00, 29.17it/s]
[1/20] Loss_D: 0.5055 Acc_D: 0.5139 Loss_G_class1: 0.1699 Loss_G_class2: 0.1753:  67%|██████▋   | 18/27 [00:25<00:12,  1.41s/it]

In [ ]:
"""
cit1 = cit_builder()
cit2 = cit_builder()
cit3 = cit_builder()

cit2._G_dict['P'].to('cpu')

#for p in cit2._G_dict['P'].parameters():
#    print(p)

state_dict = cit2._G_dict['P'].state_dict()
new_state_dict = copy.deepcopy(state_dict)

for k in state_dict:
    print(state_dict[k])
"""

In [ ]:
"""
with torch.no_grad():
    
    for k in state_dict:
        #print(k)
        new_state_dict[k] = 2*state_dict[k]
        #print(new_state_dict[k])
    cit2._G_dict['P'].load_state_dict(new_state_dict)
    

"""
    

In [ ]:
"""
state_dict = cit2._G_dict['P'].state_dict()
for k in state_dict:
    print(state_dict[k])
"""

In [ ]:
cit1 = cit_builder()
cit2 = cit_builder()
cit3 = cit_builder()

for cit in [cit1, cit2, cit3]:
    state_dict = cit._G_dict['P'].state_dict()
    k = "block1.0.weight"
    print(state_dict[k][0][0][0])

aggregator = shfl.federated_aggregator.FedAvgAggregator()
aggregated_weights = aggregator.aggregate_weights([cit1.get_model_params(), cit2.get_model_params(), cit3.get_model_params()])

cit1.set_model_params(aggregated_weights)

print("bbb")
state_dict = cit1._G_dict['P'].state_dict()
k = "block1.0.weight"
print(state_dict[k][0][0][0])


In [ ]:

cit1 = cit_builder()
cit2 = cit_builder()
cit3 = cit_builder()
k = "conv1.weight"
state_dict = cit1._classifier.state_dict()
state_dict[k][0][0][0][0] = 3
cit1._classifier.load_state_dict(state_dict)

k = "conv1.weight"
for cit in [cit1, cit2, cit3]:
    state_dict = cit._classifier.state_dict()
    print(state_dict[k][0][0][0])

state_dict[k][0][0][0][0] = 3
cit1._classifier.load_state_dict(state_dict)
    
aggregator = shfl.federated_aggregator.FedAvgAggregator()
agg = aggregator.aggregate_weights([cit1.get_model_params(), cit2.get_model_params(), cit3.get_model_params()])
cit1.set_model_params(agg)

print("bbb")
state_dict = cit1._classifier.state_dict()
k = "conv1.weight"
print(state_dict[k][0][0][0])
